# Imports

In [1]:
from langchain_ollama import OllamaLLM # Para levantar el modelo LLama3.1
from langchain_community.document_loaders import PyPDFDirectoryLoader #Para Cargar los PDFs
from langchain.text_splitter import RecursiveCharacterTextSplitter # Para hacer los CHUNKS
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings # Para hacer el embeddings de los CHUNKS
from langchain_chroma import Chroma# Para trabajar con la BD vectorial
from langchain.prompts import PromptTemplate # Para hacer el template del prompt del modelo LLama
from langchain.chains import RetrievalQA # Para hacer las preguntas al modelo y obtener las respuestas
from langchain_core.documents.base import Document # Para hacer la concatenación de documentos

In [2]:
import os

# Parámetros

In [3]:
CHUNK_SIZE=2000
CHUNK_OVERLAP=500

# Consideraciones
* Se tomó la decisión de que todos los documentos van en una sola colección

# Modelo
* El modelo es el llama3.1
* Debe de estar installado el modelo y ollama.
* [Descargar Ollama](https://ollama.com/)
* Descargar el modelo en la terminal luego de descargar OLLAMA `ollama pull llama3.1` **Pesa 4GB**


In [85]:
llm = OllamaLLM(model="llama3.1")
llm.invoke("Hola, quien eres?")

'Hola! Soy un modelo de lenguaje artificial creado por Meta, y estoy aquí para ayudarte con cualquier cosa que necesites. No tengo una identidad personal en el sentido clásico, pero puedo interactuar contigo como si fuera una persona. Me llamo Llama (Large Language Model Meta AI), pero puedes llamarme como quieras. ¿En qué puedo ayudarte hoy?'

# Cargar los Documentos (RAG)

In [5]:
direc = './PDF/'
loader=PyPDFDirectoryLoader(direc)

In [6]:
pdfs = loader.load() # Aquí están todos los PDF

# Splitter

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

### Hacer los Chunks

In [49]:
chunks=text_splitter.split_documents(pdfs)

In [90]:
chunks

[Document(metadata={'source': 'PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf', 'page': 0}, page_content='Universidad \nCatólica del Norte \nREGLAMENTO GENERAL \nDE DOCENCIA DE PREGRADO \nJUNIO 2024'),
 Document(metadata={'source': 'PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf', 'page': 1}, page_content='ÍNDICE \nTÍTULO | Disposiciones generale \nTÍTULO II De la calidad de estudiante \nTÍTULO IIl Admisión y Matricula \nTÍTULO IV Estructura y Organización Curriculal ... 7 \nTÍTULO V Sistema de Créditos Transferibles (SCT) \nTÍTULO VI Inscripción y renuncia de actividades curriculare \nTÍTULO VIl Evaluaciones y calificaciones \nTÍTULO Vill Asignaturas por Tutoría y EXAMEN ..o 18 \nTÍTULO IX Interrupción de estudios s 19 \nTÍTULO X Eliminación y Renuncia \nTÍTULO XI Cambios de carrera o de programa y cambios de sede. \nTÍTULO XIl Traslado de Universidad \nTÍTULO XIl Movilidad estudiantil e 27 \nTÍTULO XV Egreso y Titulación ...t 30 \nArtículo transitorio ...O aA 

In [10]:
chunks[0]

Document(metadata={'source': 'PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf', 'page': 0}, page_content='Universidad \nCatólica del Norte \nREGLAMENTO GENERAL \nDE DOCENCIA DE PREGRADO \nJUNIO 2024')

In [11]:
chunks[1]

Document(metadata={'source': 'PDF\\Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf', 'page': 1}, page_content='ÍNDICE \nTÍTULO | Disposiciones generale \nTÍTULO II De la calidad de estudiante \nTÍTULO IIl Admisión y Matricula \nTÍTULO IV Estructura y Organización Curriculal ... 7 \nTÍTULO V Sistema de Créditos Transferibles (SCT) \nTÍTULO VI Inscripción y renuncia de actividades curriculare \nTÍTULO VIl Evaluaciones y calificaciones \nTÍTULO Vill Asignaturas por Tutoría y EXAMEN ..o 18 \nTÍTULO IX Interrupción de estudios s 19 \nTÍTULO X Eliminación y Renuncia \nTÍTULO XI Cambios de carrera o de programa y cambios de sede. \nTÍTULO XIl Traslado de Universidad \nTÍTULO XIl Movilidad estudiantil e 27 \nTÍTULO XV Egreso y Titulación ...t 30 \nArtículo transitorio ...O aA 31')

# Hacer el embedding

## Obtener el modelo de embedding desde HuggingFace

In [12]:
embed_model = FastEmbedEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\claud\Desktop\Cursos\LearningLangChain\thisEnvironment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]


# Chroma


### Creación de la base de datos

- Solo se hizo el embedding del documento general de docencia de pregrado(Hay que arreglar concatenar los documentos)

In [13]:
vs = Chroma.from_documents(
    documents=chunks,
    embedding=embed_model,
    persist_directory="chroma_db_dir",  # Local mode with in-memory storage only
    collection_name="reglamentos_UCN"
)

In [14]:
vectorstore = Chroma(embedding_function=embed_model,
                     persist_directory="./chroma_db_dir/",
                     collection_name="reglamentos_UCN")

In [15]:
retriever=vectorstore.as_retriever(search_kwargs={'k': 10})

### Hacer el Template Del Promt

In [87]:
from langchain.prompts import PromptTemplate

custom_prompt_template = """ 
Usa la siguiente información para responder las preguntas del usuario, 
- Eres un chatbot en la Universidad Católica Del Norte - Campus Guayacan, en la ciudad de Coquimbo.
- Debes responder con fundamento, normalmente los usuarios esperaran respuestas en base a los artículos que tienen el texto y trata de entrar en detalle sobre el mismo.
- Acompaña tu respuesta cuando sea necesario con información que pueda necesitar el alumno, siempre en base al documento.
- Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.

Contexto: {context}
Pregunta: {question}

responde siempre en español a no ser que te pidan lo contrario
Respuesta útil:

"""
prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=['context', 'question'])

In [88]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True,
                                 chain_type_kwargs={"prompt": prompt})

## Prompts De prueba

In [89]:
def full_response(response):
    best_chunk=response['source_documents'][0]

    file_name = best_chunk.metadata['file_path'].split('\\')[-1].split('/')[-1]

    page_number = int(best_chunk.metadata['page']) + 1

    full_response =response['result'] + f'\nLos detalles de esta respuesta fueron encontrados en el documento "{file_name}", en la página número {page_number}.'
    print(full_response)

### Reglamento-General-de-Docencia-de-Pregrado-2024-2

In [79]:
response = qa.invoke({"query": "¿Que es un estudiante regular?"})
full_response(response)


Un estudiante regular es aquel que se encuentra cursando actividades académicas en la Universidad Católica del Norte, conservando su calidad de estudiante regulera, y que no tiene ninguna categoría especial como son el intercambio saliente o entrante, ni es un estudiante especial. Esto implica que cumple con los requisitos y procesos establecidos por la universidad para ser considerado un estudiante regular, incluyendo el pago de matrícula y la participación en el proceso anual de matrícula dentro de los plazos establecidos.
Los detalles de esta respuesta fueron encontrados en el documento "Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf", en la página número 5.


In [80]:
response = qa.invoke({"query": "¿Que es un estudiante egresado?"})
full_response(response)

Un estudiante egresado es un alumno que ha finalizado sus estudios en una institución educativa, como una universidad o escuela técnica. Es decir, alguien que ya ha completado su carrera o programa académico y ha obtenido su título de graduación.
Los detalles de esta respuesta fueron encontrados en el documento "Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf", en la página número 6.


In [81]:
response = qa.invoke({"query": "¿Que es un estudiante egresado?"})
full_response(response)

Lo siento, pero no hay una definición específica del término "estudiante egresado" en el texto proporcionado. Sin embargo, podemos inferir que un estudiante egresado es alguien que ha completado su carrera o programa académico y ha obtenido su título de grado.

En el contexto de los artículos que se proporcionan, parece estar hablando sobre estudiantes que están cursando sus estudios en la universidad. Por lo tanto, un estudiante egresado podría ser aquel que ya no está matriculado en la institución debido a la finalización de su carrera o programa.
Los detalles de esta respuesta fueron encontrados en el documento "Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf", en la página número 6.


In [82]:
response = qa.invoke({"query": "¿Que es una actividad curricular?"})
full_response(response)

Una actividad curricular son las actividades académicas que realiza un estudiante dentro de un plan de estudios, bajo la supervisión directa de uno o más profesores, y pueden incluir asignaturas, prácticas, seminarios, proyectos, memorias u otras formas de aprendizaje previstas en el plan de estudios.
Los detalles de esta respuesta fueron encontrados en el documento "Reglamento-General-de-Docencia-de-Pregrado-2024-2.pdf", en la página número 29.


In [84]:
response = qa.invoke({"query": "¿Cuales son las condiciones para inscribir una actividad curricular?"})
full_response(response)

Según el texto proporcionado, existen varias condiciones para inscribir una actividad curricular. A continuación, se presentan los prerrequisitos establecidos:

1. **Por asignatura**: El estudiante debe tener aprobada(s) la(s) asignatura(s) requisito(s) de acuerdo al Plan de Estudios vigente.
2. **Por dispersión**: El estudiante puede inscribir asignaturas de hasta dos niveles superiores a su nivel actual.
3. **Por créditos**: El estudiante debe cumplir con los créditos definidos en el Plan de Estudios vigente para poder inscribir asignaturas.
4. **Por nivel**: El estudiante debe cumplir con el nivel mínimo establecido en el Plan de Estudios vigente para poder inscribir asignaturas.

Además, si un estudiante tiene alguna excepción o situación especial que no cumple con estos prerrequisitos, puede solicitar la omisión de algún prerrequisito a la Jefatura de Carrera o Programa, quien evaluará la situación curricular del estudiante para su aprobación.
Los detalles de esta respuesta fueron

{'query': '¿Cuales son las condiciones para inscribir una actividad curricular?',
 'result': 'Según el texto proporcionado, existen varias condiciones para inscribir una actividad curricular. A continuación, se presentan los prerrequisitos establecidos:\n\n1. **Por asignatura**: El estudiante debe tener aprobada(s) la(s) asignatura(s) requisito(s) de acuerdo al Plan de Estudios vigente.\n2. **Por dispersión**: El estudiante puede inscribir asignaturas de hasta dos niveles superiores a su nivel actual.\n3. **Por créditos**: El estudiante debe cumplir con los créditos definidos en el Plan de Estudios vigente para poder inscribir asignaturas.\n4. **Por nivel**: El estudiante debe cumplir con el nivel mínimo establecido en el Plan de Estudios vigente para poder inscribir asignaturas.\n\nAdemás, si un estudiante tiene alguna excepción o situación especial que no cumple con estos prerrequisitos, puede solicitar la omisión de algún prerrequisito a la Jefatura de Carrera o Programa, quien eval

### Reglamento-permanencia-UCN.Nuevo_.-D4-2023-salida

In [46]:
response = qa.invoke({"query": "¿Que es considerado una infracción leve?"})
response['result']

'No se especifica explícitamente qué es considerado como una infracción leve en el texto proporcionado. Sin embargo, se mencionan condiciones y procedimientos relacionados con la evaluación académica, renuncia de actividades y calificaciones parciales, pero no se establecen explicitadamente las consecuencias o sanciones por incumplir estas normas como "infracciones leves".'